## Channels
|**N**          |0  |1  |2  |3  |4  | 5 | 6 | 7 | 8 | 9 |
|---------------|---|---|---|---|---|---|---|---|---|---|
|**Sentinel-2A**|B02|B03|B04|B05|B06|B07|B08|B8A|B11|B12|
|**Standard**   |B  |G  |R  |RE1|RE2|RE3|N  |N2 |S1 |S2 |

## Classes:
|**N**        |1  |2  |3  |4  | 5 |
|-|-|-|-|-|-|
|**seg. mask**|open water|settlements|bair soil|forest|grassland|

## Indexes:
|short|long|type|formula|
|-|-|-|-|
LSWI|Land Surface Water Index|water|(N - S1)/(N + S1)

In [ ]:
from LandcoverDataset import LandcoverDataset as LD

from utils import *
from DataVis import *
from Model import CNN, CNN_load

In [ ]:
TRAIN_PATH = "datasets/USA/train/"
IMG_TRAIN = TRAIN_PATH + "/images/"
MASK_TRAIN = TRAIN_PATH + "/masks/"
l_train = len(get_filenames_in_dir(IMG_TRAIN))

VALID_PATH = "datasets/USA/valid/"
IMG_VALID = VALID_PATH + "/images/"
MASK_VALID = VALID_PATH + "/masks/"
l_valid = len(get_filenames_in_dir(IMG_VALID))

In [ ]:
print(f"Train samples: {l_train}")
print(f"Test samples: {l_valid}")

In [ ]:
BATCH = 16

dl_train = LD(
    img_path = IMG_TRAIN, 
    mask_path= MASK_TRAIN,
    n_random = 300,
    batch_size = BATCH
)
dl_valid = LD(
    img_path=IMG_VALID, 
    mask_path=MASK_VALID,
    n_random = 50,
    batch_size = BATCH,
    transforms = False
)

In [ ]:
img, mask = dl_train.rand_samp_names()
print(tif_info(img))
print(tif_info(mask))

In [ ]:
print(f"Images: {dl_train[0][0].shape}")
print(f"Masks:  {dl_train[0][1].shape}")

In [ ]:
dl_train.plot_sample(0, index="(N - S1)/(N + S1)")

In [ ]:
dl_train.getinfo()

In [ ]:
import torch.nn as nn

INPUT_CHANNELS = 10
N_CLASSES = 5

layers = [
    nn.Conv2d(INPUT_CHANNELS, 16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, N_CLASSES, kernel_size=1)
]

model = CNN(layers)

In [ ]:
num_epochs = 5

In [ ]:
learning_rate = 0.005

In [ ]:
model.train(dl_train, dl_valid, num_epochs, learning_rate)

In [ ]:
from random import choice as c
dl_valid.plot_prediction(model, c(range(50)))

In [ ]:
img, mask = ds_train.rand_samp_names()
prediction = model.predict(img)
compare_prediction(img, mask, prediction)

In [ ]:
m_name = f"ne={epochs_sum}, hl={hidden_layers}"
model.pickle(f"models/{m_name}.torch")

In [ ]:
model.unpickle("models/model.torch")

In [ ]:
model = CNN_load("models/model.torch")